In [1]:
# 导入相关包
%matplotlib inline
# 绘图包
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 定义文件名
ACTION_201602_FILE = "data/JData_Action_201602.csv"
ACTION_201603_FILE = "data/JData_Action_201603.csv"
# ACTION_201603_EXTRA_FILE = "data/JData_Action_201603_extra.csv"
ACTION_201604_FILE = "data/JData_Action_201604.csv"
COMMENT_FILE = "data/JData_Comment.csv"
PRODUCT_FILE = "data/JData_Product.csv"
USER_FILE = "data/JData_User.csv"
NEW_USER_FILE = "data/JData_User_New.csv"
USER_TABLE_FILE = "data/user_table.csv"
BUY_USER_LIST_FILE = "data/buy_user_list.csv"
PROTENTIAL_USER_RECORD = "data/protential_user_record.csv"

In [5]:
#寻找有购买记录的用户/商品信息
def buy_user_in_batch_data(fname,chunk_size=100000):
    reader = pd.read_csv(fname,header=0,iterator=True)
    chunks = []
    loop = True
    while loop:
        try:
            chunk = reader.get_chunk(chunk_size)[['user_id','sku_id','type']]
            chunk = chunk[chunk['type'] == 4]
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print('Iteration is stopped')
            
    df_ac = pd.concat(chunks,ignore_index=True)
    df_ac = df_ac[['user_id','sku_id']]
    
    return df_ac

In [3]:
def find_buy_user():
    df_ac = []
    df_ac.append(buy_user_in_batch_data(fname=ACTION_201602_FILE))
    df_ac.append(buy_user_in_batch_data(fname=ACTION_201603_FILE))
    df_ac.append(buy_user_in_batch_data(fname=ACTION_201604_FILE))
    
    df_ac = pd.concat(df_ac, ignore_index=True)
    #丢弃同一用户购买同一商品记录
    df_ac.drop_duplicates()
    #写入文件
    df_ac.to_csv(BUY_USER_LIST_FILE, index=False)

In [6]:
find_buy_user()

Iteration is stopped
Iteration is stopped
Iteration is stopped


In [11]:
# 在一个文件中寻找与给定的user-item对有关的所有记录
def ui_record_in_batch_data(fname, ui_pair, chunk_size=100000):
    reader = pd.read_csv(fname, header=0, iterator=True)
    chunks = []
    loop = True
    while loop:
        try:
            chunk = reader.get_chunk(chunk_size)[
                ["user_id", "sku_id", "time", "type"]]
            for index,row in ui_pair.iterrows():
                user_id = row['user_id']
                sku_id = row['sku_id']
                df_temp = chunk[(chunk['user_id'] == user_id) & (chunk['sku_id'] == sku_id)]
                chunks.append(df_temp)
            print(len(chunk.index))
        except StopIteration:
            loop = False
            print("Iteration is stopped")

#     df_ac = pd.concat(chunks, ignore_index=True)
    
#     df = []
#     for index, row in ui_pair.iterrows():
#         usr_id = row["user_id"]
#         sku_id = row["sku_id"]

#         # 寻找与user-item对有关的所有记录
#         df.append(df_ac[(df_ac["user_id"] == usr_id) &
#                         (df_ac["sku_id"] == sku_id)])

    df = pd.concat(chunks, ignore_index=True)
    return df

In [12]:
# apply功能函数：根据一个user-item对的所有记录，计算当前是否是高潜用户
def more_than_a_day(group):
    # 最后一次购买该商品的日期
    last_buy_day = max(group[group["type"] == 4]["date"])
    # 最早与该商品发生交互的日期
    earliest_behave_day = min(group["date"])
    
    # 如果间隔不小于１天，则认为是高潜用户
    if (last_buy_day - earliest_behave_day).days > 0:
        # 字段potential_flag代表是否是高潜用户
        group["potential_flag"] = 1
    else:
        group["potential_flag"] = 0

    return group

In [13]:
# 寻找高潜用户，并将相关行为记录写入文件
def find_potential_user():
    # 有购买行为的user-item对
    ui_pair = pd.read_csv(BUY_USER_LIST_FILE, header=0)

    df_ac = []
    df_ac.append(ui_record_in_batch_data(ACTION_201602_FILE, ui_pair))
    df_ac.append(ui_record_in_batch_data(fname=ACTION_201603_FILE))
#     df_ac.append(ui_record_in_batch_data(fname=ACTION_201603_EXTRA_FILE))
    df_ac.append(ui_record_in_batch_data(fname=ACTION_201604_FILE))

    df_ac = pd.concat(df_ac, ignore_index=True)
    # 丢弃重复的
    df_ac = df_ac.drop_duplicates()
    
    # 增加日期属性
    df_ac['date'] = pd.to_datetime(df_ac['time']).dt.date
    df_ac = df_ac.groupby(["user_id", "sku_id"]).apply(more_than_a_day)
    
    # 找出高潜用户
    df_ac = df_ac[df_ac["potential_flag"] == 1]
    # 写入文件
    df_ac.to_csv(PROTENTIAL_USER_RECORD, index=False)

In [14]:
find_potential_user()

100000
100000
100000
100000
100000
100000


KeyboardInterrupt: 